In [20]:
import cv2
import numpy as np
import os
from tqdm import tqdm
import glob

def compute_similarities(input_video_path, num_frames=200, verbose=False):
    # Open the input video
    cap = cv2.VideoCapture(input_video_path)

    # Check if video opened successfully
    if not cap.isOpened():
        if verbose:
            print("Error: Could not open video.")
        return None, None, None, None

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    ret, frame = cap.read()
    if not ret:
        if verbose:
            print("Failed to read the first frame of the video.")
        cap.release()
        return None, None, None, None

    height, width, channels = frame.shape
    center_y = height // 2
    center_x = width // 2

    # Initialize array to store frames
    frames_array = []

    if verbose:
        print("Reading frames...")

    # Read the specified number of frames
    frame_count = 0
    while frame_count < num_frames:
        if frame is None:
            if verbose:
                print(f"Reached end of video at frame {frame_count}.")
            break
        frames_array.append(frame)
        ret, frame = cap.read()
        frame_count += 1

    cap.release()
    num_frames = len(frames_array)

    if verbose:
        print(f"Total frames read: {num_frames}")

    # Convert frames list to numpy array
    frames_array = np.array(frames_array, dtype=np.uint8)  # Shape: (num_frames, height, width, 3)

    # Reshape frames to (height * width, num_frames * channels)
    if verbose:
        print("Processing pixel time series...")
    frames_reshaped = frames_array.transpose(1, 2, 0, 3).reshape(height * width, num_frames * channels)

    # Get the time series of the center pixel
    center_index = center_y * width + center_x
    center_pixel_vector = frames_reshaped[center_index, :]

    # Compute Euclidean distances
    if verbose:
        print("Calculating Euclidean distances...")
    diffs = frames_reshaped.astype(np.float32) - center_pixel_vector.astype(np.float32)
    distances = np.linalg.norm(diffs, axis=1)

    # Normalize distances to get similarity (0 to 1)
    if verbose:
        print("Normalizing distances...")
    max_distance = distances.max()
    if max_distance == 0:
        if verbose:
            print("Max distance is zero. All pixels are identical to the center pixel.")
        similarity = np.ones_like(distances)
    else:
        similarity = 1 - (distances / max_distance)

    # Reshape similarity back to image shape
    similarity_image = similarity.reshape(height, width)

    return frames_array, similarity_image, fps, (height, width, channels), (center_x, center_y)

def process_frames_for_threshold(frames_array, similarity_image, threshold, output_video_path, fps, frame_size, center_coords, dot_size=10):
    height, width, channels = frame_size
    center_x, center_y = center_coords
    num_frames = frames_array.shape[0]

    # Create mask for pixels based on threshold
    white_mask = (similarity_image >= threshold).astype(np.float32)[:, :, np.newaxis]  # Shape: (height, width, 1)

    # Normalize frames to [0,1] for blending with the white mask
    blended_frames_float = frames_array.astype(np.float32) / 255.0
    white_frame = np.ones((height, width, 3), dtype=np.float32)  # Fully white pixel overlay

    # Blend the frames with white based on the mask
    final_frames = blended_frames_float * (1.0 - white_mask) + white_frame * white_mask
    final_frames = np.clip(final_frames * 255, 0, 255).astype(np.uint8)

    # Initialize VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for i in range(num_frames):
        frame = final_frames[i]

        # Add red dot at the center of the frame
        cv2.circle(frame, (center_x, center_y), dot_size, (0, 0, 255), 1)

        # Add the threshold text
        text = f"threshold: {threshold:.2f}"
        position = (width - 300, 50)  # Top-right position
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        color = (0, 0, 0)  # Black color for text
        thickness = 2
        cv2.putText(frame, text.lower(), position, font, font_scale, color, thickness, lineType=cv2.LINE_AA)

        out.write(frame)

    out.release()

def main():
    input_video_path = 'cab_ride_trimmed.mkv'
    output_dir = 'videos'
    num_frames = 200
    verbose = False

    # Compute similarities once
    frames_array, similarity_image, fps, frame_size, center_coords = compute_similarities(
        input_video_path, num_frames=num_frames, verbose=verbose)

    if frames_array is None:
        print("Failed to compute similarities.")
        return

    # Create multiple videos for different thresholds
    threshold_values = [0.99, 0.98, 0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

    for idx, threshold in enumerate(tqdm(threshold_values, desc="Creating threshold videos")):
        output_path = os.path.join(output_dir, f'euclidean_annotated_video_threshold_{threshold:.2f}.mp4')
        current_dot_size = max(1, 10 - idx)  # Decrease dot size, ensuring it doesn't go below 1
        process_frames_for_threshold(
            frames_array, similarity_image, threshold, output_path, fps, frame_size, center_coords, dot_size=current_dot_size)

    # Stitch all videos together
    print("Stitching all videos together...")
    output_combined_path = os.path.join(output_dir, 'combined_thresholds_video.mp4')
    video_files = sorted(glob.glob(os.path.join(output_dir, 'euclidean_annotated_video_threshold_*.mp4')))

    # Get properties from the first video
    cap = cv2.VideoCapture(video_files[0])
    if not cap.isOpened():
        print("Error: Could not open the first video for stitching.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_combined = cv2.VideoWriter(output_combined_path, fourcc, fps, (width, height))

    for video_file in tqdm(video_files, desc="Stitching videos"):
        cap = cv2.VideoCapture(video_file)
        if not cap.isOpened():
            print(f"Error: Could not open video {video_file}")
            continue

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            out_combined.write(frame)
        cap.release()

    out_combined.release()
    print(f"Combined video saved as {output_combined_path}.")

    # Delete the non-combined videos
    print("Deleting non-combined videos...")
    for video_file in video_files:
        try:
            os.remove(video_file)
            print(f"Deleted {video_file}")
        except OSError as e:
            print(f"Error: Could not delete {video_file}. {e}")

if __name__ == '__main__':
    main()


Creating threshold videos: 100%|███████████████████████████████████████████████████████| 14/14 [02:43<00:00, 11.65s/it]


Stitching all videos together...


Stitching videos: 100%|████████████████████████████████████████████████████████████████| 14/14 [00:24<00:00,  1.74s/it]

Combined video saved as videos\combined_thresholds_video.mp4.
Deleting non-combined videos...
Deleted videos\euclidean_annotated_video_threshold_0.10.mp4
Deleted videos\euclidean_annotated_video_threshold_0.20.mp4
Deleted videos\euclidean_annotated_video_threshold_0.30.mp4
Deleted videos\euclidean_annotated_video_threshold_0.40.mp4
Deleted videos\euclidean_annotated_video_threshold_0.50.mp4
Deleted videos\euclidean_annotated_video_threshold_0.60.mp4
Deleted videos\euclidean_annotated_video_threshold_0.70.mp4
Deleted videos\euclidean_annotated_video_threshold_0.75.mp4
Error: Could not delete videos\euclidean_annotated_video_threshold_0.80.mp4. [WinError 32] The process cannot access the file because it is being used by another process: 'videos\\euclidean_annotated_video_threshold_0.80.mp4'
Deleted videos\euclidean_annotated_video_threshold_0.85.mp4
Deleted videos\euclidean_annotated_video_threshold_0.90.mp4
Deleted videos\euclidean_annotated_video_threshold_0.95.mp4
Deleted videos\eucli